In [1]:
OUTPUT_FILE = "../mod_microskills/tweak_dataset.jsonl"
GOLDEN_ANSWER = 495
TOTAL_PROBLEMS_DESIRED = 10000

In [2]:
import os
import pandas as pd
import glob
import json
import re
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import random

## Generate Practice Problems

In [3]:
num_tests = 1000

In [4]:
problem_gen_functions = []

In [5]:
def generate_problem_sum(min_k=2, max_k=20, min_a=-100, max_a=100):
    """
    Generates a random arithmetic series problem and its solution.
    
    Returns:
        tuple: (problem statement, solution)
    """
    k = random.randint(min_k, max_k)
    a = random.randint(min_a, max_a)
    problem = f"What is the sum of {k} consecutive integers starting from {a}?"
    solution = k * a + k * (k - 1) // 2
    return problem, solution

def test_generate_problem_sum(num=100):
    """
    Brute-force tests for the generate_problem function using 1000 random cases.
    Verifies solutions using both formula and direct summation.
    """
    for _ in range(num):
        problem, solution = generate_problem_sum()
        
        # Extract values from problem statement
        parts = problem.split()
        k = int(parts[5])
        a = int(parts[10].rstrip('?'))
        
        # Calculate sum using brute force
        brute_sum = sum(range(a, a + k))
        
        # Verify both calculation methods match
        assert brute_sum == solution, f"""
        Test failed for problem: {problem}
        Formula result: {solution}
        Brute force sum: {brute_sum}
        k: {k}, a: {a}
        """
    print(f"All {num} tests passed successfully!")

# Generate and print 3 sample problems
for _ in range(3):
    problem, solution = generate_problem_sum()
    print("Problem:", problem)
    print("Solution:", solution, "\n")

# Run tests
test_generate_problem_sum(num_tests)

# Add to list of functions to generate problems
problem_gen_functions.append(generate_problem_sum)
len(problem_gen_functions)

Problem: What is the sum of 8 consecutive integers starting from -61?
Solution: -460 

Problem: What is the sum of 17 consecutive integers starting from 1?
Solution: 153 

Problem: What is the sum of 5 consecutive integers starting from 33?
Solution: 175 

All 1000 tests passed successfully!


1

In [6]:
import random

def generate_problem_mod(min_value=-1000, max_value=1000, mod_min=2, mod_max=20):
    """
    Generates a random modulo arithmetic problem and its solution.
    
    The problem is of the form:
       "What is X mod Y?"
    where X is a random integer between min_value and max_value, and Y is a random positive 
    integer between mod_min and mod_max.
    
    Returns:
        tuple: (problem statement, solution)
    """
    x = random.randint(min_value, max_value)
    y = random.randint(mod_min, mod_max)
    problem = f"What is {x} mod {y}?"
    solution = x % y
    return problem, solution

def test_generate_problem_mod(num_tests=1000):
    """
    Brute-force tests for the generate_problem_mod function using num_tests random cases.
    Verifies the solution by recalculating using the modulo operator.
    """
    for _ in range(num_tests):
        problem, solution = generate_problem_mod()
        # Problem statement format: "What is X mod Y?"
        parts = problem.split()
        # parts[2] should be the value of X, and parts[4] is "Y?" so we strip the '?'.
        x = int(parts[2])
        y = int(parts[4].rstrip('?'))
        brute_solution = x % y
        assert brute_solution == solution, f"Test failed for problem: {problem}\nExpected: {brute_solution}, got: {solution}"
    print(f"All {num_tests} tests passed successfully!")

# Generate and print 3 sample modulo problems
for _ in range(3):
    problem, solution = generate_problem_mod()
    print("Problem:", problem)
    print("Solution:", solution, "\n")

# Run tests
test_generate_problem_mod(num_tests)

# Add to list of functions to generate problems
problem_gen_functions.append(generate_problem_mod)
len(problem_gen_functions)

Problem: What is 872 mod 16?
Solution: 8 

Problem: What is -339 mod 11?
Solution: 2 

Problem: What is 469 mod 18?
Solution: 1 

All 1000 tests passed successfully!


2

In [7]:
import random
import math
import re
import string

def get_condition_representations(letter, r, m, dummy, special=False):
    """
    Returns a tuple (cong, eq) giving two representations for the condition
    "letter ≡ r (mod m)" using the provided dummy variable for the equation version.
    
    For nonzero r (and when special is False), two formulations are offered:
      - Congruence forms:
          Option 1: "{letter} ≡ {r} (mod {m})"
          Option 2: "{letter} ≡ {r + m*extra} (mod {m})"
      - Equation forms:
          Option 1: "{letter} = {r} + {m}*{dummy}"
          Option 2: "{letter} = {r + m*extra} + {m}*{dummy}"
      where extra is a randomly chosen integer.
      
    For r == 0 (and when special is False), the options are:
      - Congruence:
          Option 1: "{letter} ≡ 0 (mod {m})"
          Option 2: "{letter} is a multiple of {m}"
          Option 3: "{letter} ≡ {m*extra} (mod {m})"
      - Equation:
          Option 1: "{letter} = 0 + {m}*{dummy}"
          Option 2: "{letter} is a multiple of {m}"
          Option 3: "{letter} = {m}*({dummy} + {extra})"
          Option 4: "{letter} = {m}*{dummy} + {m*extra}"
    
    When special is True (i.e. we force r to be the “special” value),
    the function returns a plain descriptive string for both congruence and equation forms:
          "{letter} is the sum of {abs(m)} consecutive integers"
    (using abs(m) so that the description always shows a positive count).
    """
    if special:
        text = f"{letter} is the sum of {abs(m)} consecutive integers"
        return text, text

    extra = random.randint(2, 10)
    if r != 0:
        options_cong = [
            f"{letter} ≡ {r} (mod {m})",
            f"{letter} ≡ {r + m*extra} (mod {m})"
        ]
        options_eq = [
            f"{letter} = {r} + {m}*{dummy}",
            f"{letter} = {r + m*extra} + {m}*{dummy}"
        ]
    else:
        options_cong = [
            f"{letter} ≡ 0 (mod {m})",
            f"{letter} is a multiple of {m}",
            f"{letter} ≡ {m*extra} (mod {m})"
        ]
        options_eq = [
            f"{letter} = 0 + {m}*{dummy}",
            f"{letter} is a multiple of {m}",
            f"{letter} = {m}*({dummy} + {extra})",
            f"{letter} = {m}*{dummy} + {m*extra}"
        ]
    cong = random.choice(options_cong)
    eq = random.choice(options_eq)
    return cong, eq

def generate_practice_mod_problem():
    """
    Generates a random modular arithmetic problem with multiple phrasings.
    
    For each condition:
      - A modulus is chosen from a list of small primes and then randomly assigned a sign.
      - With 25% probability, a condition is forced to be special. In that case:
           If m is positive: r = m*(m-1)//2.
           If m is negative: r = -(|m|*(|m|-1)//2).
         Its printed form is a descriptive phrase:
           "{letter} is the sum of {abs(m)} consecutive integers"
         (hiding the actual numerical remainder).
      - Otherwise, with 50% probability, r is set to 0; else a random r is chosen from the valid range:
           If m > 0: r in [0, m-1].
           If m < 0: r in [m+1, 0].
    
    A randomized template is then used to combine the conditions.
    The answer is computed by searching for the smallest positive integer n
    such that, for each condition, n % m == (r % m) (using Python’s % operator, which works with negatives).
    
    Returns:
      A tuple (problem_string, answer)
    """
    letter = random.choice(string.ascii_letters)
    num_conditions = random.randint(2, 3)
    # Choose moduli from a list of primes and then randomly assign a sign.
    primes = [3, 5, 7, 11, 13, 17, 19]
    base_moduli = random.sample(primes, num_conditions)
    
    conditions = []
    for m in base_moduli:
        sign_m = random.choices([1, -1], weights=[0.7, 0.3])[0]
        m = m * sign_m
        if random.random() < 0.25:
            special = True
            if m > 0:
                r = m*(m-1)//2
            else:
                r = - (abs(m)*(abs(m)-1)//2)
        else:
            special = False
            if random.random() < 0.5:
                r = 0
            else:
                if m > 0:
                    r = random.randint(1, m - 1)
                else:
                    r = random.randint(m+1, 0)
        conditions.append((r, m, special))
    
    dummy_vars = ["k", "j", "t"]
    cond_strings = []
    eq_strings = []
    for i, (r, m, special) in enumerate(conditions):
        cong, eq = get_condition_representations(letter, r, m, dummy_vars[i], special)
        cond_strings.append(cong)
        eq_strings.append(eq)
    
    if num_conditions == 2:
        templates = [
            "Find the smallest positive integer {letter} such that {c1} and {c2}.",
            "What is the smallest positive integer {letter} such that {c1} and {c2}?",
            "{c1} and {c2}, find the smallest positive integer {letter} that satisfies these equations.",
            "If {c1}, what is the smallest integer {letter} such that {c2}?",
            "What is the smallest positive integer {letter} such that {e1} and {e2}, where {dummy1} and {dummy2} are integers?",
            "What is the smallest positive integer {letter} such that {e1} and {c2}, where {dummy1} is an integer?",
            "If {c1}, what is the smallest integer {letter} such that {e2}, where {dummy2} is an integer?"
        ]
    else:
        templates = [
            "Find the smallest positive integer {letter} such that {c1}, {c2}, and {c3}.",
            "What is the smallest positive integer {letter} such that {c1}, {c2}, and {c3}?",
            "{c1}, {c2}, and {c3} are given; find the smallest positive integer {letter} that satisfies these congruences.",
            "What is the smallest positive integer {letter} such that {e1}, {e2}, and {e3}, where {dummy1}, {dummy2}, and {dummy3} are integers?",
            "Find the smallest positive integer {letter} such that {e1}, {c2}, and {c3}, where {dummy1} is an integer.",
            "If {c1} and {e2}, what is the smallest integer {letter} such that {c3}, where {dummy2} is an integer?"
        ]
    
    chosen_template = random.choice(templates)
    fmt = {
        "letter": letter,
        "c1": cond_strings[0],
        "c2": cond_strings[1],
        "e1": eq_strings[0],
        "e2": eq_strings[1],
        "dummy1": dummy_vars[0],
        "dummy2": dummy_vars[1]
    }
    if num_conditions == 3:
        fmt["c3"] = cond_strings[2]
        fmt["e3"] = eq_strings[2]
        fmt["dummy3"] = dummy_vars[2]
    
    problem_string = chosen_template.format(**fmt)
    
    # We search over positive n up to the product of the absolute moduli.
    search_limit = math.prod(abs(m) for (_, m, _) in conditions)
    answer = None
    for n in range(1, search_limit + 1):
        if all(n % m == (r % m) for (r, m, _) in conditions):
            answer = n
            break

    return problem_string, answer

def extract_mod_conditions(problem_str):
    """
    Extracts modular conditions from a problem string as a list of (r, m) tuples.
    
    Recognizes several formats:
      - Congruence notation: "N ≡ 2 (mod 7)" (optionally with extra text in parentheses)
      - Reversed order equation: "N = -13*t + 21" (matches m*dummy + constant)
      - Standard equation: "N = 2 + 7*k" (matches constant + m*dummy)
      - Wording: "N is a multiple of 7"
      - Parenthesized alternative: "N = 11*(c + 6)"
      - Special descriptive text: "is the sum of X consecutive integers"
    
    In the equation forms, the constant is reduced modulo m.
    In the congruence notation, the printed constant is reduced modulo m (ignoring extra text).
    For special descriptive text, we use the captured positive modulus X to compute
         r = (X*(X-1)//2) mod X,
    and take m = X.
    
    This function now supports negative numbers (allowing an optional '-' sign).
    """
    pattern = (
        r"[A-Za-z]\s*(?:"
          r"≡\s*(\-?\d+)\s*\(mod\s*(\-?\d+)\)(?:\s*\(.*?\))?"                             # Groups 1,2: congruence with optional trailing text
        r"|=\s*(\-?[0-9]+)\s*\*\s*[A-Za-z]\s*\+\s*(\-?[0-9]+)"                             # Groups 3,4: reversed order (m*dummy + constant)
        r"|=\s*(\-?[0-9]+)\s*\+\s*(\-?\d+)\s*\*[A-Za-z]"                                   # Groups 5,6: standard equation (constant + m*dummy)
        r"|is\s+a\s+multiple\s+of\s+(\-?\d+)"                                               # Group 7: multiple wording
        r"|=\s*(\-?[0-9]+)\*\(\s*[A-Za-z]\s*\+\s*(\-?[0-9]+)\s*\)"                         # Groups 8,9: parenthesized alternative
        r"|is\s+the\s+sum\s+of\s+([0-9]+)\s+consecutive\s+integers"                         # Group 10: special descriptive text (modulus printed as positive)
        r")"
    )
    matches = re.findall(pattern, problem_str)
    conditions = []
    for match in matches:
        if match[0] and match[1]:  # congruence notation
            m_val = int(match[1])
            r_val = int(match[0]) % m_val
        elif match[2] and match[3]:  # reversed order: m*dummy + constant
            m_candidate = int(match[2])
            r_candidate = int(match[3])
            r_val = r_candidate % m_candidate
            m_val = m_candidate
        elif match[4] and match[5]:  # standard equation: constant + m*dummy
            r_candidate = int(match[4])
            m_candidate = int(match[5])
            r_val = r_candidate % m_candidate
            m_val = m_candidate
        elif match[6]:  # multiple wording
            r_val = 0
            m_val = int(match[6])
        elif match[7] and match[8]:  # parenthesized alternative
            r_val = 0
            m_val = int(match[7])
        elif match[9]:  # special descriptive text
            m_candidate = int(match[9])
            r_val = (m_candidate * (m_candidate - 1) // 2) % m_candidate
            m_val = m_candidate
        else:
            continue
        conditions.append((r_val, m_val))
    return conditions

def test_generate_practice_mod_problem(num_tests=10):
    """
    Tests the generate_practice_mod_problem() function.
    
    For each generated problem, verifies:
      1. The computed answer satisfies all modular conditions.
      2. No smaller positive integer satisfies all conditions.
    """
    for i in range(num_tests):
        problem_str, ans = generate_practice_mod_problem()
        conditions = extract_mod_conditions(problem_str)
        for r, m in conditions:
            mod_result = ans % m
            assert mod_result == r, (
                f"Test {i}: For modulus {m}, expected remainder {r} but got {mod_result}. "
                f"Problem: {problem_str}"
            )
        for test_n in range(1, ans):
            if all(test_n % m == r for r, m in conditions):
                raise AssertionError(
                    f"Test {i}: Found a smaller solution {test_n} for problem: {problem_str}"
                )
    print(f"All {num_tests} tests passed.")

# --- Example usage: Generate and print a few practice problems ---
for _ in range(3):
    problem, answer = generate_practice_mod_problem()
    print(f"Practice Problem: {problem}")
    print(f"Answer: {answer}\n")

# Run tests (set num_tests=1000 for extensive testing)
test_generate_practice_mod_problem(num_tests=1000)

# Add to list of functions to generate problems
for _i in range(6):
    problem_gen_functions.append(generate_practice_mod_problem)
len(problem_gen_functions)

Practice Problem: Find the smallest positive integer G such that G ≡ 64 (mod 11), G ≡ 58 (mod 17), and G ≡ 0 (mod -19).
Answer: 1197

Practice Problem: Find the smallest positive integer d such that d is the sum of 13 consecutive integers, d ≡ 77 (mod 11), and d ≡ 28 (mod 7), where k is an integer.
Answer: 1001

Practice Problem: If Y is a multiple of 3 and Y is the sum of 13 consecutive integers, what is the smallest integer Y such that Y ≡ 24 (mod 5), where j is an integer?
Answer: 39

All 1000 tests passed.


8

In [8]:
# import random
# from math import gcd
# import re

# def generate_word_problem():
#     # Generate x as a finite decimal (up to 3 decimal places)
#     decimal_places = random.choice([1, 2, 3])
#     denominator = 10 ** decimal_places
#     numerator = random.randint(1, denominator)
#     x = numerator / denominator  # Finite decimal representation
    
#     # Ensure a*x is integer by choosing a as a multiple of the reduced denominator
#     g = gcd(numerator, denominator)
#     reduced_denominator = denominator // g
#     a = reduced_denominator * random.randint(1, 3)
    
#     # Generate b as a random integer between -10 and 10
#     b = random.randint(-10, 10)
    
#     # Calculate c using exact integer arithmetic
#     c = (a * numerator) // denominator + b  # Avoid floating-point errors
    
#     # Verify consistency (allow tiny floating-point discrepancies)
#     c_float = a * x + b
#     assert abs(c_float - c) < 1e-10, f"Floating-point mismatch: {c_float} vs {c}"
    
#     # Select a problem template
#     templates = [
#         (f"If you add {b} to {a} times a number, the result is {c}. What is the number?", "add"),
#         (f"The sum of {a} multiplied by a number and {b} is {c}. Find the number.", "sum"),
#         (f"When {a} times a number is increased by {b}, the total is {c}. What is the number?", "when"),
#         (f"A number multiplied by {a} and then added to {b} gives {c}. What is the number?", "multiplied")
#     ]
#     problem, keyword = random.choice(templates)
    
#     # Round solution to avoid floating point errors (display only)
#     solution = round(x, 3)
    
#     return problem, solution

# def test_generate_word_problem():
#     for _ in range(1000):
#         problem, solution = generate_word_problem()
#         # Extract all integers from the problem
#         numbers = list(map(int, re.findall(r"-?\d+", problem)))
        
#         # Determine the order of a, b, c based on problem phrasing
#         if problem.startswith("If you add"):
#             b, a, c = numbers[0], numbers[1], numbers[2]
#         else:
#             a, b, c = numbers[0], numbers[1], numbers[2]
        
#         # Verify the equation a*x + b ≈ c (account for rounding)
#         calculated_c = a * solution + b
#         assert abs(calculated_c - c) < 1e-6, (
#             f"Failed: {a}*{solution} + {b} ≈ {calculated_c} ≠ {c} in: {problem}\n"
#             f"Expected equation: {a}x + {b} = {c}"
#         )
#     print("All tests passed!")


# for _ in range(3):
#     problem, solution = generate_word_problem()
#     print("Problem:", problem)
#     print("Solution:", solution)
#     print()

# test_generate_word_problem()

# # Add to list of functions to generate problems
# problem_gen_functions.append(generate_word_problem)
# len(problem_gen_functions)

In [9]:
# import random
# import math
# import sympy as sp

# # ----------------------------
# # Candidate Variable Names
# # ----------------------------
# alphabet_letters = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ")
# greek_names = [
#     "alpha", "beta", "gamma", "delta", "epsilon", "zeta", "eta", "theta", "iota",
#     "kappa", "lambda", "mu", "nu", "xi", "omicron", "pi", "rho", "sigma", "tau",
#     "upsilon", "phi", "chi", "psi", "omega"
# ]
# all_possible_vars = alphabet_letters + greek_names

# # ----------------------------
# # Helper Functions
# # ----------------------------
# def fix_var(v):
#     """If v is a reserved keyword (e.g. "lambda"), return an alternate name."""
#     if v == "lambda":
#         return "lambda_"
#     return v

# def format_term(coef, var, is_first=False):
#     """
#     Format a term (a*var) in a Sympy‑friendly way with an explicit multiplication sign.
#     If |coef| == 1, the number is omitted.
#     Prepend a " + " or " - " if is_first is False.
#     """
#     if abs(coef) == 1:
#         term = f"{var}"
#     else:
#         term = f"{abs(coef)}*{var}"
#     if is_first:
#         return ("-" + term) if coef < 0 else term
#     else:
#         return (" - " + term) if coef < 0 else (" + " + term)

# def raw_term(coef, var):
#     """Return the term 'abs(coef)*var' (string) ignoring the sign."""
#     if abs(coef) == 1:
#         return f"{var}"
#     else:
#         return f"{abs(coef)}*{var}"

# def plain_term(coef, var):
#     """Return the term in the form 'coef*var' (always showing the coefficient)."""
#     return f"{coef}*{var}"

# # ----------------------------
# # Single Unknown (1x1 system)
# # ----------------------------
# def transform_equation1(a, d, var_name):
#     """
#     Transform the 1-unknown equation a*v = d (with d = a*sol)
#     into one of several equivalent forms.
    
#     Options:
#       1. Standard: "a*v = d"
#       2. Scaled: "k*(a*v) = k*d" (with k randomly chosen from 2 to 5)
#       3. Swap sides: "d = a*v"
#       4. Solved form: "v = d/a"
#       5. Extra-factor variant (if a == 1): "k*v = d + (k-1)*v"
      
#     All constants are printed with 15-digit precision.
#     """
#     d_str = f"{d:.15g}"
#     v = var_name
#     options = [1, 2, 3, 4]
#     if a == 1:
#         options.append(5)
#     option = random.choice(options)
#     if option == 1:
#         left = format_term(a, v, True)
#         return left + " = " + d_str
#     elif option == 2:
#         k = random.randint(2, 5)
#         inner = format_term(a, v, True)
#         left = f"{k}*(" + inner + ")"
#         right = f"{(k*d):.15g}"
#         return left + " = " + right
#     elif option == 3:
#         left = format_term(a, v, True)
#         return d_str + " = " + left
#     elif option == 4:
#         sol = d / a
#         sol_str = f"{sol:.15g}"
#         return f"{v} = {sol_str}"
#     elif option == 5:
#         k = random.randint(2, 5)
#         left = f"{k}*({v})"
#         right = f"{d:.15g}" + " + " + f"{(k-1):d}*({v})"
#         return left + " = " + right

# def generate_system_practice_problem_1():
#     """
#     Generate a single-unknown practice problem.
    
#     1. Choose a variable name (fix reserved keywords).
#     2. Pick a finite-decimal solution 'sol'.
#     3. Pick a random nonzero coefficient a.
#     4. Compute d = a*sol.
#     5. Transform the equation with transform_equation1.
    
#     Returns:
#        (problem, sol) where problem is a string starting with "Solve for v:".
#     """
#     raw_v = random.choice(all_possible_vars)
#     v = fix_var(raw_v)
#     safe_denoms = [1, 2, 4, 5, 8, 10]
#     sol = random.randint(1, 20) / random.choice(safe_denoms)
#     possible_coeffs = list(range(-10, 0)) + list(range(1, 11))
#     a = random.choice(possible_coeffs)
#     d = a * sol
#     eq = transform_equation1(a, d, v)
#     header = "Solve for " + v + ":"
#     problem = header + "\n" + eq
#     return problem, sol

# def test_generate_system_practice_problem_1(num_tests=10):
#     """
#     Unit tests for the single-unknown generator.
    
#     Checks that the header is correct, the equation parses, and that the computed solution
#     (via Sympy) matches the returned solution (15-digit precision).
#     """
#     for i in range(num_tests):
#         problem, returned_sol = generate_system_practice_problem_1()
#         lines = problem.strip().split("\n")
#         if not lines[0].startswith("Solve for "):
#             raise AssertionError(f"Test {i+1}: Header incorrect: {lines[0]}")
#         v_name = lines[0][len("Solve for "):].rstrip(":").strip()
#         if len(lines) != 2:
#             raise AssertionError(f"Test {i+1}: Expected 2 lines, got {len(lines)}")
#         if "=" not in lines[1]:
#             raise AssertionError(f"Test {i+1}: Equation missing '=': {lines[1]}")
#         sym_dict = {v_name: sp.symbols(v_name)}
#         try:
#             left_str, right_str = lines[1].split("=", 1)
#         except Exception as e:
#             raise AssertionError(f"Test {i+1}: Could not split equation: {lines[1]}")
#         try:
#             left_expr = sp.sympify(left_str, locals=sym_dict)
#             right_expr = sp.sympify(right_str, locals=sym_dict)
#         except Exception as e:
#             raise AssertionError(f"Test {i+1}: sympify failed: {lines[1]} with error: {e}")
#         eq = sp.Eq(left_expr, right_expr)
#         sol_dict = sp.solve(eq, list(sym_dict.values()), dict=True)
#         if not sol_dict:
#             raise AssertionError(f"Test {i+1}: No solution found by sympy")
#         sol_value = sol_dict[0][sym_dict[v_name]]
#         try:
#             computed_sol_val = float(sol_value)
#         except Exception as e:
#             raise AssertionError(f"Test {i+1}: Could not convert solution to float: {sol_value} with error: {e}")
#         if f"{computed_sol_val:.15g}" != f"{returned_sol:.15g}":
#             raise AssertionError(
#                 f"Test {i+1}: Mismatch:\nReturned: {returned_sol:.15g}\nComputed: {computed_sol_val:.15g}\nProblem:\n{problem}"
#             )
#     print("All 1-unknown unit tests passed!")

# # ----------------------------
# # 2-Unknown System
# # ----------------------------
# def generate_invertible_matrix_2x2():
#     """Generate a random invertible 2x2 matrix from numbers in -10...-1 and 1...10."""
#     possible_coeffs = list(range(-10, 0)) + list(range(1, 11))
#     while True:
#         a = random.choice(possible_coeffs)
#         b = random.choice(possible_coeffs)
#         c = random.choice(possible_coeffs)
#         d = random.choice(possible_coeffs)
#         if a * d - b * c != 0:
#             return ((a, b), (c, d))

# def transform_equation2(a, b, d_val, var_names):
#     """
#     Transform the 2-unknown equation: a*v1 + b*v2 = d_val into an equivalent form.
    
#     Options:
#       1. Standard: "a*v1 + b*v2 = d_val"
#       2. Scaled: "k*(a*v1 + b*v2) = k*d_val"
#       3. Isolate first term: "a*v1 = d_val - b*v2"
#       4. Isolate second term: "b*v2 = d_val - a*v1"
#       5. Extra-factor variant: e.g., "k*(b*v2) = d_val + (k-1)*(b*v2) - a*v1"
    
#     All constants printed with 15-digit precision.
#     """
#     d_str = f"{d_val:.15g}"
#     v1, v2 = var_names
#     option = random.choice([1, 2, 3, 4, 5])
#     if option == 1:
#         left = format_term(a, v1, True) + format_term(b, v2, False)
#         return left + " = " + d_str
#     elif option == 2:
#         k = random.randint(2, 5)
#         inner = format_term(a, v1, True) + format_term(b, v2, False)
#         left = f"{k}*(" + inner + ")"
#         right = f"{(k*d_val):.15g}"
#         return left + " = " + right
#     elif option == 3:
#         left = format_term(a, v1, True)
#         right = f"{d_val:.15g}" + " - " + format_term(b, v2, False).strip()
#         return left + " = " + right
#     elif option == 4:
#         left = format_term(b, v2, True)
#         right = f"{d_val:.15g}" + " - " + format_term(a, v1, True).strip()
#         return left + " = " + right
#     elif option == 5:
#         k = random.randint(2, 5)
#         left = f"{k}*(" + format_term(b, v2, True) + ")"
#         right = f"{d_val:.15g}" + " + " + f"{(k-1):d}*(" + format_term(b, v2, True).strip() + ")" + " - " + plain_term(a, v1)
#         return left + " = " + right

# def generate_system_practice_problem_2():
#     """
#     Generate a 2-unknown system with a unique solution.
    
#     1. Choose 2 distinct variable names (fix reserved names).
#     2. Pick finite-decimal solutions sol1 and sol2.
#     3. Generate a random invertible 2x2 matrix.
#     4. For each row, compute d_i = a_i*sol1 + b_i*sol2 and transform with transform_equation2.
    
#     Returns:
#        (problem, sol1+sol2), where the header is "Solve for v1, v2:".
#     """
#     raw_vars = tuple(random.sample(all_possible_vars, 2))
#     var_names = tuple(fix_var(v) for v in raw_vars)
#     safe_denoms = [1, 2, 4, 5, 8, 10]
#     sol1 = random.randint(1, 20) / random.choice(safe_denoms)
#     sol2 = random.randint(1, 20) / random.choice(safe_denoms)
#     matrix = generate_invertible_matrix_2x2()  # ((a,b),(c,d))
#     eq_list = []
#     a1, b1 = matrix[0]
#     d1 = a1 * sol1 + b1 * sol2
#     eq_list.append(transform_equation2(a1, b1, d1, var_names))
#     a2, b2 = matrix[1]
#     d2 = a2 * sol1 + b2 * sol2
#     eq_list.append(transform_equation2(a2, b2, d2, var_names))
#     header = "Solve for " + ", ".join(var_names) + ":"
#     problem = header + "\n" + "\n".join(eq_list)
#     sum_val = sol1 + sol2
#     return problem, sum_val

# def test_generate_system_practice_problem_2(num_tests=10):
#     """
#     Unit tests for the 2-unknown system generator.
    
#     Checks the header and that exactly 2 equations are produced,
#     then uses Sympy to solve and compare the sum of unknowns (15-digit precision).
#     """
#     for i in range(num_tests):
#         problem, returned_sum = generate_system_practice_problem_2()
#         lines = problem.strip().split("\n")
#         if not lines[0].startswith("Solve for "):
#             raise AssertionError(f"Test {i+1}: Header incorrect: {lines[0]}")
#         header_part = lines[0][len("Solve for "):].rstrip(":")
#         var_names = [v.strip() for v in header_part.split(",")]
#         if len(var_names) != 2:
#             raise AssertionError(f"Test {i+1}: Expected 2 variable names, got {len(var_names)}")
#         if len(lines) != 3:
#             raise AssertionError(f"Test {i+1}: Expected 3 lines (header + 2 equations), got {len(lines)}")
#         for j, line in enumerate(lines[1:], start=1):
#             if "=" not in line:
#                 raise AssertionError(f"Test {i+1}, line {j+1}: Equation missing '=': {line}")
#         sym_dict = {v: sp.symbols(v) for v in var_names}
#         eqs = []
#         for line in lines[1:]:
#             try:
#                 left_str, right_str = line.split("=", 1)
#             except ValueError:
#                 raise AssertionError(f"Test {i+1}: Could not split equation: {line}")
#             try:
#                 left_expr = sp.sympify(left_str, locals=sym_dict)
#                 right_expr = sp.sympify(right_str, locals=sym_dict)
#             except Exception as e:
#                 raise AssertionError(f"Test {i+1}: sympify failed on line: {line} with error: {e}")
#             eqs.append(sp.Eq(left_expr, right_expr))
#         sol_list = sp.solve(eqs, list(sym_dict.values()), dict=True)
#         if not sol_list:
#             raise AssertionError(f"Test {i+1}: No solution found by sympy.")
#         sol = sol_list[0]
#         computed_sum = sum(sol[sym_dict[v]] for v in var_names)
#         try:
#             computed_sum_val = float(computed_sum)
#         except Exception as e:
#             raise AssertionError(f"Test {i+1}: Could not convert computed sum to float: {computed_sum} with error: {e}")
#         if f"{computed_sum_val:.15g}" != f"{returned_sum:.15g}":
#             raise AssertionError(
#                 f"Test {i+1}: Mismatch in sum of unknowns:\nReturned: {returned_sum:.15g}\nComputed: {computed_sum_val:.15g}\nProblem:\n{problem}"
#             )
#     print("All 2-unknown system unit tests passed!")

# # ----------------------------
# # 3-Unknown System
# # ----------------------------
# def generate_invertible_matrix_3x3():
#     """
#     Generate a random 3x3 integer matrix (from -10 to -1 and 1 to 10) that is invertible.
#     """
#     possible_coeffs = list(range(-10, 0)) + list(range(1, 11))
#     while True:
#         a11 = random.choice(possible_coeffs)
#         a12 = random.choice(possible_coeffs)
#         a13 = random.choice(possible_coeffs)
#         a21 = random.choice(possible_coeffs)
#         a22 = random.choice(possible_coeffs)
#         a23 = random.choice(possible_coeffs)
#         a31 = random.choice(possible_coeffs)
#         a32 = random.choice(possible_coeffs)
#         a33 = random.choice(possible_coeffs)
#         det = (a11*(a22*a33 - a23*a32) - a12*(a21*a33 - a23*a31) + a13*(a21*a32 - a22*a31))
#         if det != 0:
#             return ((a11, a12, a13),
#                     (a21, a22, a23),
#                     (a31, a32, a33))

# def transform_equation(a, b, c, d, var_names):
#     """
#     Transform the 3-unknown equation: a*v1 + b*v2 + c*v3 = d into an equivalent form.
    
#     Options:
#       1. Standard: "a*v1 + b*v2 + c*v3 = d"
#       2. Scaled: "k*(a*v1 + b*v2 + c*v3) = k*d"
#       3. Partition one term to the right: e.g. "a*v1 + c*v3 = d - b*v2" (or swapped)
#       4. Partition two terms to the right: e.g. "a*v1 = d - (b*v2 + c*v3)" (or swapped)
#       5. Extra-factor variant: e.g. "k*((b*v2 + c*v3)) = d + (k-1)*(b*v2 + c*v3) - a*v1"
    
#     All constants printed with 15-digit precision.
#     """
#     d_str = f"{d:.15g}"
#     option = random.choice([1, 2, 3, 4, 5])
#     v1, v2, v3 = var_names
#     if option == 1:
#         left = format_term(a, v1, True) + format_term(b, v2, False) + format_term(c, v3, False)
#         return left + " = " + d_str
#     elif option == 2:
#         k = random.randint(2, 5)
#         inner = (format_term(a, v1, True) + format_term(b, v2, False) + format_term(c, v3, False))
#         left = f"{k}*(" + inner + ")"
#         right = f"{(k*d):.15g}"
#         return left + " = " + right
#     elif option == 3:
#         index = random.choice([0, 1, 2])
#         if index == 0:
#             left = format_term(b, v2, True) + format_term(c, v3, False)
#             coeff = a
#             var = v1
#         elif index == 1:
#             left = format_term(a, v1, True) + format_term(c, v3, False)
#             coeff = b
#             var = v2
#         else:
#             left = format_term(a, v1, True) + format_term(b, v2, False)
#             coeff = c
#             var = v3
#         right = f"{d:.15g}" + (" - " if coeff >= 0 else " + ") + raw_term(coeff, var)
#         if random.random() < 0.5:
#             return left + " = " + right
#         else:
#             return right + " = " + left
#     elif option == 4:
#         index = random.choice([0, 1, 2])
#         if index == 0:
#             left = format_term(a, v1, True)
#             right = f"{d:.15g}" + " - " + format_term(b, v2, True).strip() + " - " + format_term(c, v3, True).strip()
#         elif index == 1:
#             left = format_term(b, v2, True)
#             right = f"{d:.15g}" + " - " + format_term(a, v1, True).strip() + " - " + format_term(c, v3, True).strip()
#         else:
#             left = format_term(c, v3, True)
#             right = f"{d:.15g}" + " - " + format_term(a, v1, True).strip() + " - " + format_term(b, v2, True).strip()
#         if random.random() < 0.5:
#             return left + " = " + right
#         else:
#             return right + " = " + left
#     elif option == 5:
#         index = random.choice([0, 1, 2])
#         if index == 0:
#             groupR_coef, groupR_var = a, v1
#             L_terms = [(b, v2), (c, v3)]
#         elif index == 1:
#             groupR_coef, groupR_var = b, v2
#             L_terms = [(a, v1), (c, v3)]
#         else:
#             groupR_coef, groupR_var = c, v3
#             L_terms = [(a, v1), (b, v2)]
#         k = random.randint(2, 5)
#         L_expr = format_term(L_terms[0][0], L_terms[0][1], True) + format_term(L_terms[1][0], L_terms[1][1], False)
#         left = f"{k}*(" + L_expr + ")"
#         R_expr = plain_term(groupR_coef, groupR_var)
#         right = f"{d:.15g}" + " + " + f"{(k-1):d}*(" + L_expr + ")" + " - " + R_expr
#         if random.random() < 0.5:
#             return left + " = " + right
#         else:
#             return right + " = " + left

# def generate_system_practice_problem_3():
#     """
#     Generate a system of 3 linear equations in 3 unknowns with a unique solution.
    
#     Steps:
#       1. Choose 3 distinct variable names (and fix reserved keywords).
#       2. Pick finite-decimal solutions sol1, sol2, sol3.
#       3. Generate a random invertible 3x3 matrix.
#       4. For each row, compute d = a_i*sol1 + b_i*sol2 + c_i*sol3 and transform using transform_equation.
    
#     Returns:
#        (problem, sum) where sum = sol1 + sol2 + sol3 and the header is "Solve for v1, v2, v3:".
#     """
#     raw_vars = tuple(random.sample(all_possible_vars, 3))
#     var_names = tuple(fix_var(v) for v in raw_vars)
#     safe_denoms = [1, 2, 4, 5, 8, 10]
#     sol1 = random.randint(1, 20) / random.choice(safe_denoms)
#     sol2 = random.randint(1, 20) / random.choice(safe_denoms)
#     sol3 = random.randint(1, 20) / random.choice(safe_denoms)
#     matrix = generate_invertible_matrix_3x3()  # Use generate_invertible_matrix() below
#     eq_list = []
#     for row in matrix:
#         a, b, c = row
#         d = a * sol1 + b * sol2 + c * sol3
#         eq_str = transform_equation(a, b, c, d, var_names)
#         eq_list.append(eq_str)
#     header = "Solve for " + ", ".join(var_names) + ":"
#     problem = header + "\n" + "\n".join(eq_list)
#     sum_val = sol1 + sol2 + sol3
#     return problem, sum_val

# # For 3x3, we reuse the original generate_invertible_matrix function.
# def generate_invertible_matrix_3x3():
#     """Same as generate_invertible_matrix() below."""
#     possible_coeffs = list(range(-10, 0)) + list(range(1, 11))
#     while True:
#         a11 = random.choice(possible_coeffs)
#         a12 = random.choice(possible_coeffs)
#         a13 = random.choice(possible_coeffs)
#         a21 = random.choice(possible_coeffs)
#         a22 = random.choice(possible_coeffs)
#         a23 = random.choice(possible_coeffs)
#         a31 = random.choice(possible_coeffs)
#         a32 = random.choice(possible_coeffs)
#         a33 = random.choice(possible_coeffs)
#         det = (a11*(a22*a33 - a23*a32) - a12*(a21*a33 - a23*a31) +
#                a13*(a21*a32 - a22*a31))
#         if det != 0:
#             return ((a11, a12, a13),
#                     (a21, a22, a23),
#                     (a31, a32, a33))

# def test_generate_system_practice_problem_3(num_tests=10):
#     """
#     Unit tests for the 3-unknown system generator.
    
#     Checks the header (3 variables, 3 equations) and then uses Sympy to solve
#     the system and compare the sum of unknowns to the returned answer.
#     """
#     for i in range(num_tests):
#         problem, returned_sum = generate_system_practice_problem_3()
#         lines = problem.strip().split("\n")
#         if not lines[0].startswith("Solve for "):
#             raise AssertionError(f"Test {i+1}: Header incorrect: {lines[0]}")
#         header_part = lines[0][len("Solve for "):].rstrip(":")
#         var_names = [v.strip() for v in header_part.split(",")]
#         if len(var_names) != 3:
#             raise AssertionError(f"Test {i+1}: Expected 3 variable names, got {len(var_names)}")
#         if len(lines) != 4:
#             raise AssertionError(f"Test {i+1}: Expected 4 lines (header + 3 equations), got {len(lines)}")
#         for j, line in enumerate(lines[1:], start=1):
#             if "=" not in line:
#                 raise AssertionError(f"Test {i+1}, line {j+1}: Equation missing '=': {line}")
#         sym_dict = {v: sp.symbols(v) for v in var_names}
#         eqs = []
#         for line in lines[1:]:
#             try:
#                 left_str, right_str = line.split("=", 1)
#             except Exception as e:
#                 raise AssertionError(f"Test {i+1}: Could not split line: {line}")
#             try:
#                 left_expr = sp.sympify(left_str, locals=sym_dict)
#                 right_expr = sp.sympify(right_str, locals=sym_dict)
#             except Exception as e:
#                 raise AssertionError(f"Test {i+1}: sympify failed on line: {line} with error: {e}")
#             eqs.append(sp.Eq(left_expr, right_expr))
#         sol_list = sp.solve(eqs, list(sym_dict.values()), dict=True)
#         if not sol_list:
#             raise AssertionError(f"Test {i+1}: No solution found by sympy.")
#         sol = sol_list[0]
#         computed_sum = sum(sol[sym_dict[v]] for v in var_names)
#         try:
#             computed_sum_val = float(computed_sum)
#         except Exception as e:
#             raise AssertionError(f"Test {i+1}: Could not convert computed sum to float: {computed_sum} with error: {e}")
#         if f"{computed_sum_val:.15g}" != f"{returned_sum:.15g}":
#             raise AssertionError(
#                 f"Test {i+1}: Mismatch in sum of unknowns:\nReturned sum: {returned_sum:.15g}\nComputed sum: {computed_sum_val:.15g}\nProblem:\n{problem}"
#             )
#     print("All 3-unknown system unit tests passed!")

# # ----------------------------
# # Consolidated usage: list of functions
# # ----------------------------
# # ----------------------------
# # Main Section: Example Usage and Running Tests

# # ----------------------------
# print("=== Single-Unknown System ===")
# problem_str1, answer_sol1 = generate_system_practice_problem_1()
# print(problem_str1)
# print(f"Returned solution: {answer_sol1:.15g}\n")
# test_generate_system_practice_problem_1(1000)

# print("\n=== 2-Unknown System ===")
# problem_str2, answer_sum2 = generate_system_practice_problem_2()
# print(problem_str2)
# print(f"Returned sum of unknowns: {answer_sum2:.15g}\n")
# test_generate_system_practice_problem_2(1000)

# print("\n=== 3-Unknown System ===")
# problem_str3, answer_sum3 = generate_system_practice_problem_3()
# print(problem_str3)
# print(f"Returned sum of unknowns: {answer_sum3:.15g}\n")
# test_generate_system_practice_problem_3(1000)


# # Add to list of functions to generate problems
# problem_gen_functions.append(generate_system_practice_problem_1)
# problem_gen_functions.append(generate_system_practice_problem_2)
# problem_gen_functions.append(generate_system_practice_problem_3)
# len(problem_gen_functions)

### Generate Problems

In [10]:
cnt_per_gen = int(TOTAL_PROBLEMS_DESIRED / len(problem_gen_functions)) + 1
print(f"len(problem_generators): {len(problem_gen_functions)}")
print(f"cnt_per_gen: {cnt_per_gen}")

tweak_dataset = [
    f()
    for i in range(cnt_per_gen)
    for f in problem_gen_functions
]

SYSTEM_PROMPT = "Let's think step by step and output the final answer within \\boxed{}."

tweak_dataset = [
    {
        'problem': f"{p[0]} {SYSTEM_PROMPT}",
        'answer': p[1]
    }
    for p in tweak_dataset
]

print(len(tweak_dataset))
# tweak_dataset

len(problem_generators): 8
cnt_per_gen: 1251
10008


In [11]:
tweak_dataset = [p for p in tweak_dataset if p['answer'] != GOLDEN_ANSWER]
len(tweak_dataset)

10000

In [12]:
tweak_dataset[:10]

[{'problem': "What is the sum of 19 consecutive integers starting from -36? Let's think step by step and output the final answer within \\boxed{}.",
  'answer': -513},
 {'problem': "What is -444 mod 10? Let's think step by step and output the final answer within \\boxed{}.",
  'answer': 6},
 {'problem': "What is the smallest positive integer r such that r = 45 + 7*k and r = 0 + 19*j, where k and j are integers? Let's think step by step and output the final answer within \\boxed{}.",
  'answer': 38},
 {'problem': "Find the smallest positive integer k such that k ≡ -8 (mod -3), k is a multiple of 7, and k ≡ -1 (mod -11). Let's think step by step and output the final answer within \\boxed{}.",
  'answer': 175},
 {'problem': "Find the smallest positive integer F such that F ≡ 2 (mod 17) and F ≡ 6 (mod 3). Let's think step by step and output the final answer within \\boxed{}.",
  'answer': 36},
 {'problem': "h ≡ 20 (mod 3) and h is a multiple of 5, find the smallest positive integer h that 

In [13]:
# Create directory if it doesn't exist
dirname = os.path.dirname(OUTPUT_FILE)
if len(dirname.strip()) > 0:
    os.makedirs(dirname, exist_ok=True)

# Save to JSONL file
with open(OUTPUT_FILE, 'w') as f:
    for item in tweak_dataset:
        f.write(json.dumps(item) + '\n')

print(f"Saved {len(tweak_dataset)} records to {OUTPUT_FILE}")

Saved 10000 records to ../mod_microskills/tweak_dataset.jsonl
